In [1]:
# Import libraries
import pytesseract
from pdf2image import convert_from_path
from PIL import Image, ImageDraw, ImageFont
import fitz  # PyMuPDF
from googletrans import Translator
import warnings
warnings.filterwarnings('ignore')

# Specify the path to tesseract executable if it's not in your PATH
pytesseract.pytesseract.tesseract_cmd = r"C:\Users\sengu\Downloads\tesseract-ocr-w64-setup-5.3.4.20240503.exe"  # Update this path as necessary

# Extract text from image
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image)
    return text

# Extract text from PDF
def extract_text_from_pdf(pdf_path):
    pages = convert_from_path(pdf_path)
    text = ""
    for page in pages:
        text += pytesseract.image_to_string(page)
    return text

# Translate text
def translate_text(text, src_lang='en', tgt_lang='fr'):
    translator = Translator()
    translated = translator.translate(text, src=src_lang, dest=tgt_lang)
    return translated.text

# Replace text in image
def replace_text_in_image(image_path, translated_text):
    image = Image.open(image_path)
    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()
    draw.text((10, 10), translated_text, font=font, fill="black")
    # Save the translated image in the same directory
    output_image_path = "translated_image.png"
    image.save(output_image_path)
    print(f"Translated image saved to {output_image_path}")

# Replace text in PDF
def replace_text_in_pdf(pdf_path, translated_text):
    doc = fitz.open(pdf_path)
    for page in doc:
        text = page.get_text()
        page.insert_text((10, 10), translated_text)
    # Save the translated PDF in the same directory
    output_pdf_path = "translated_document.pdf"
    doc.save(output_pdf_path)
    print(f"Translated PDF saved to {output_pdf_path}")

# Main function
def main(file_path, file_type, src_lang='en', tgt_lang='fr'):
    if file_type == 'image':
        text = extract_text_from_image(file_path)
    elif file_type == 'pdf':
        text = extract_text_from_pdf(file_path)
    else:
        raise ValueError("Unsupported file type. Use 'image' or 'pdf'.")

    translated_text = translate_text(text, src_lang, tgt_lang)

    if file_type == 'image':
        replace_text_in_image(file_path, translated_text)
    elif file_type == 'pdf':
        replace_text_in_pdf(file_path, translated_text)

In [2]:
main(r"C:\Users\sengu\Downloads\Get_Started_With_Smallpdf.pdf", 'pdf', src_lang='en', tgt_lang='fr')

OSError: [WinError 740] The requested operation requires elevation